In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_i = pd.read_csv('/kaggle/input/nba-injuries-2010-2018/injuries_2010-2020.csv')
#df_i are injuries
df_p = pd.read_csv('/kaggle/input/nba-players-data/all_seasons.csv', index_col=0)
#df_p are the players

In [ ]:
df_p.describe()

In [ ]:
df_i = df_i.drop('Acquired', axis=1)
df_i = df_i.loc[df_i['Relinquished'].notnull()]

In [ ]:
# Add a year column to match the season
df_i['year'] = pd.DatetimeIndex(df_i['Date']).year
df_i['month'] = pd.DatetimeIndex(df_i['Date']).month

In [ ]:
df_i.loc[971]

In [ ]:
df_i.drop('adj_season', axis=1, errors='ignore')

In [ ]:
df_i.drop('adj_season', axis=1, errors='ignore')
for index, row in df_i.iterrows():
    month = df_i.loc[index, 'month']
    df_i.loc[index, 'adj_season'] = str(int(df_i.loc[index, 'year']))
    if month < 7:
        #print(index, df_i.loc[index, 'Relinquished'], df_i.loc[index, 'year'], df_i.loc[index, 'month'])
        df_i.loc[index, 'adj_season'] = str(int(df_i.loc[index, 'year'] - 1))

In [ ]:
df_i

In [ ]:
df_i[df_i['month']==6]

In [ ]:
df_p['season']

In [ ]:
df_p['adj_season'] = df_p.season.str.split('-').str.get(0)

In [ ]:
boozer = df_p['player_name']=='Carlos Boozer'
df_p.loc[boozer]

In [ ]:
df_p['adj_season'] = df_p['adj_season'].astype(str)
df_i['adj_season'] = df_i['adj_season'].astype(str)

In [ ]:
df_comb= pd.merge(df_i, df_p, left_on=['Relinquished', 'adj_season'], right_on=['player_name', 'adj_season'], how='inner')

In [ ]:
klove = df_comb[df_comb['player_name']=='Kevin Love']

In [ ]:
klove

In [ ]:
df_i[df_i['Relinquished']=='Kevin Love']

In [ ]:
klove[['Date', 'player_name', 'player_height', 'player_weight', 'Notes']]

In [ ]:
klove['Notes'].str.find('groin')

In [ ]:
df_comb['is_knee_injury'] = df_comb['Notes'].str.find('knee')

In [ ]:
df_final = df_comb[df_comb['is_knee_injury']!=-1]

In [ ]:
df_final

In [ ]:
df_final.describe()

In [ ]:
df_injuries = df_final[['Date', 'Team', 'player_name', 'Notes', 'age', 'player_height', 'player_weight', 'gp', 'pts']]

In [ ]:
df_injuries['injury_count']

In [ ]:
df_injuries['injury_count']==1

In [ ]:
df_injuries[df_injuries['player_name']=='Nick Young']

In [ ]:
df_export = df_injuries.groupby("player_name").agg({"injury_count":np.sum, "player_height": np.max, "player_weight":np.max})

In [ ]:
df_export

In [ ]:
df_export.to_csv('/kaggle/working/knee_injuries.csv')

In [ ]:
df_export['is_tall']=(df_export['player_height']>198.12)

In [ ]:
df_export